In [1]:
import sys
if "/Users/raymond/code/FinalProject563" not in sys.path:
    sys.path.append("/Users/raymond/code/FinalProject563")
import os
import time
import torch
import random
import numpy as np
import pandas
import glob
import matplotlib.pyplot as plt
import pickle

import connect_four
import mcts
import connect_net
import connect_plot
import evaluation
import utils
import runner

In [2]:
os.chdir("/Users/raymond/code/FinalProject563")
iteration = 1
sim_folder = utils.get_simulation_folder(trial="test01", iteration=iteration)
sim_files = glob.glob(os.path.join(sim_folder, f"examples-{iteration}-*.pickle"))
sim_model_path = runner.get_model_path(folder=sim_folder, iteration=iteration)

In [3]:
nnet = connect_net.load_model(sim_model_path)
nnet0 = connect_net.load_model(sim_model_path)
# nnet = connect_net.ConnectNet()
# nnet0 = connect_net.ConnectNet()

ConnectNet(
  (layer1): Sequential(
    (0): Conv2d(1, 128, kernel_size=(4, 4), stride=(1, 1))
    (1): Tanh()
  )
  (fc): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 4), stride=(1, 1))
    (1): Tanh()
    (2): Flatten(start_dim=1, end_dim=-1)
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): Tanh()
    (5): Linear(in_features=128, out_features=8, bias=True)
  )
)
ConnectNet(
  (layer1): Sequential(
    (0): Conv2d(1, 128, kernel_size=(4, 4), stride=(1, 1))
    (1): Tanh()
  )
  (fc): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 4), stride=(1, 1))
    (1): Tanh()
    (2): Flatten(start_dim=1, end_dim=-1)
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): Tanh()
    (5): Linear(in_features=128, out_features=8, bias=True)
  )
)


In [4]:
# num_mcts_sims = 25
# results = []
# for j in range(15):
#     with torch.no_grad():
#         result = evaluation.competition_game(
#             player1=nnet, p1_mcts_sims=num_mcts_sims,
#             player2=nnet0, p2_mcts_sims=num_mcts_sims)
#         results.append(result)
# df_res = pandas.DataFrame([(r[0], r[1]) for r in results], columns=["Winner", "moves"])
# df_res["Winner"].value_counts()

In [5]:
examples = []
for fname in sim_files:
    with open(fname, "rb") as f:
        examples += pickle.load(f)

In [6]:
train_games, val_games = runner.split_games(examples=examples, train_fraction=0.85)

In [7]:
train_boards, train_scores, train_moves = runner.split_items(games=train_games)
print(train_boards.shape, train_scores.shape, train_moves.shape)
val_boards, val_scores, val_moves = runner.split_items(games=val_games)
print(val_boards.shape, val_scores.shape, val_moves.shape)

torch.Size([24893, 6, 7]) torch.Size([24893]) torch.Size([24893, 7])
torch.Size([4333, 6, 7]) torch.Size([4333]) torch.Size([4333, 7])


In [8]:
learning_rate = 0.001
optimizer = torch.optim.Adam(nnet.parameters(), lr=learning_rate)
losses_train = []
losses_val = []

In [9]:
def full_loss_fn(est_scores, est_probs, target_scores, target_probs):
    loss_scores = (est_scores - target_scores).square().sum()
    loss_probs = (target_probs * est_probs.log()).sum()
    loss_total = loss_scores - loss_probs
    return loss_total

In [10]:
num_mcts_sims = 32
epochs = 100

In [12]:
print("Start")
results = []
for j in range(100):
    with torch.no_grad():
        result = evaluation.competition_game(
            player1=nnet, p1_mcts_sims=num_mcts_sims,
            player2=nnet0, p2_mcts_sims=num_mcts_sims)
        results.append(result)
df_res = pandas.DataFrame([(r[0], r[1]) for r in results], columns=["Winner", "moves"])
print(df_res["Winner"].value_counts())

for i in range(epochs):
    start_time = time.time()
    v, proba = nnet(train_boards.view(-1, 1, 6, 7))
    train_loss = full_loss_fn(v, proba, train_scores, train_moves)
    losses_train.append(train_loss.item())

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    with torch.no_grad():
        v, proba = nnet(val_boards.view(-1, 1, 6, 7))
        val_loss = full_loss_fn(v, proba, val_scores, val_moves).item()
        losses_val.append(val_loss)
    if i % 5 == 0:
        print(f"EPOCH::{i}  LOSS::{val_loss:.4f}  TIME::{time.time() - start_time:.4f}")
        results = []
        for j in range(100):
            with torch.no_grad():
                result = evaluation.competition_game(
                    player1=nnet, p1_mcts_sims=num_mcts_sims,
                    player2=nnet0, p2_mcts_sims=num_mcts_sims)
                results.append(result)
        df_res = pandas.DataFrame([(r[0], r[1]) for r in results], columns=["Winner", "moves"])
        print(df_res["Winner"].value_counts())
        

Start
player1    57
player2    42
Draw        1
Name: Winner, dtype: int64
tensor(15936.2607, grad_fn=<SumBackward0>) tensor(-40824.4062, grad_fn=<SumBackward0>)
tensor(4021.3914) tensor(-7120.5083)
EPOCH::0  LOSS::11141.8994  TIME::2.5736
player1    65
player2    35
Name: Winner, dtype: int64
tensor(15899.4092, grad_fn=<SumBackward0>) tensor(-40798.0508, grad_fn=<SumBackward0>)
tensor(4025.1963) tensor(-7116.3765)
tensor(15863.1045, grad_fn=<SumBackward0>) tensor(-40772.1406, grad_fn=<SumBackward0>)
tensor(4027.6558) tensor(-7112.3271)
tensor(15826.7705, grad_fn=<SumBackward0>) tensor(-40746.7305, grad_fn=<SumBackward0>)
tensor(4031.2163) tensor(-7108.3525)
tensor(15790.4062, grad_fn=<SumBackward0>) tensor(-40721.6914, grad_fn=<SumBackward0>)
tensor(4034.6802) tensor(-7104.4375)
tensor(15754.5820, grad_fn=<SumBackward0>) tensor(-40697.0312, grad_fn=<SumBackward0>)
tensor(4037.7759) tensor(-7100.5703)
EPOCH::5  LOSS::11138.3457  TIME::2.3967
player1    56
player2    42
Draw        2
Na

KeyboardInterrupt: 

In [16]:
# connect_net.save_model(nnet, "../models/pretrain01-iter03.model")